# FedCAD: Centralized vs Federated (orchestration)

This notebook launches centralized and federated experiments via existing entry points:
- Centralized: `python train_central.py`
- Federated: `flwr run .`

Metrics tracked in W&B:
- Centralized: `train_loss`, `test_loss`, `test_acc` (already logged by `train_central.py`)
- Federated: depends on your Flower app (ensure it logs to the same W&B project/group)

Recommended seeds for a seminar: 3 (e.g., `[0, 1, 2]`). Use 5 if time permits for tighter confidence intervals.

In [ ]:
import os, sys, subprocess, shutil, time
import torch

# W&B configuration
def load_wandb_config():
    """Load WandB credentials from config file (optional for reproducibility)"""
    config = {}
    config_path = './.wandb_config'
    try:
        with open(config_path, 'r') as f:
            for line in f:
                line = line.strip()
                if line and not line.startswith('#'):
                    key, value = line.split('=', 1)
                    config[key] = value
        return config.get('WANDB_ENTITY'), config.get('WANDB_PROJECT')
    except FileNotFoundError:
        return None, None

entity, project_name = load_wandb_config()
WANDB_MODE = "online"               # set to "offline" or "disabled" if needed

# Experiment seeds (3 is seminar-friendly)
SEEDS = [0, 1, 2]

# Centralized hyperparameters
EPOCHS_CENTRAL = 10
LR_CENTRAL = 1e-3
EVAL_INTERVAL = 1

# Device
USE_GPU_FLAG = torch.cuda.is_available()
print("Device:", "cuda" if USE_GPU_FLAG else "cpu")
print("Seeds:", SEEDS)

def run_cmd(cmd_list, extra_env=None, cwd=None):
    env = os.environ.copy()
    if extra_env:
        env.update({k: str(v) for k, v in extra_env.items()})
    print("\n>>>", " ".join(cmd_list))
    print("    in:", cwd or os.getcwd())
    print("    with env overrides:", extra_env or {})
    proc = subprocess.Popen(cmd_list, cwd=cwd, env=env)
    proc.wait()
    if proc.returncode != 0:
        raise RuntimeError(f"Command failed with exit code {proc.returncode}: {' '.join(cmd_list)}")

Device: cpu
Seeds: [0, 1, 2]


## Centralized experiments

Runs `train_central.py` once per seed. The script already logs: `train_loss`, `test_loss`, `test_acc`.

Note: `train_central.py` does not expose a `--seed` flag. If you need strict reproducibility per seed, add seeding to the script; otherwise each run will use random initialization.

In [ ]:
for seed in SEEDS:
    # W&B grouping so runs are easy to compare
    env_overrides = {
        "WANDB_PROJECT": project_name,
        "WANDB_MODE": WANDB_MODE,
        "WANDB_RUN_GROUP": "centralized",
        "WANDB_TAGS": "centralized,notebook",
        "WANDB_ENTITY": entity,

        # Optional: many apps read SEED from env; train_central.py currently does not
        "SEED": seed,
    }

    cmd = [sys.executable, "train_central.py",
           "--epochs", str(EPOCHS_CENTRAL),
           "--lr", str(LR_CENTRAL),
           "--eval-interval", str(EVAL_INTERVAL),
           "--wandb-mode", WANDB_MODE]
    if USE_GPU_FLAG:
        cmd.append("--gpu")

    print(f"\n=== Centralized run, seed={seed} ===")
    run_cmd(cmd, extra_env=env_overrides, cwd=os.getcwd())
    time.sleep(1)  # small gap between runs


=== Centralized run, seed=0 ===

>>> /Users/zoe/Desktop/Fed-CAD/.venv/bin/python train_central.py --epochs 10 --lr 0.001 --eval-interval 1 --wandb-mode online
    in: /Users/zoe/Desktop/Fed-CAD
    with env overrides: {'WANDB_PROJECT': 'FedCAD', 'WANDB_MODE': 'online', 'WANDB_RUN_GROUP': 'centralized', 'WANDB_TAGS': 'centralized,notebook', 'SEED': 0}


wandb: Currently logged in as: zoematr (vae-seminar) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: setting up run qieo1jzm
wandb: Tracking run with wandb version 0.23.0
wandb: Run data is saved locally in /Users/zoe/Desktop/Fed-CAD/wandb/run-20260121_150146-qieo1jzm
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run centralized
wandb: ⭐️ View project at https://wandb.ai/vae-seminar/FedCAD
wandb: 🚀 View run at https://wandb.ai/vae-seminar/FedCAD/runs/qieo1jzm


Epoch 1/10 | train_loss=0.6063 | test_loss=0.5468 | test_acc=0.7636
Epoch 2/10 | train_loss=0.5763 | test_loss=0.4901 | test_acc=0.7636
Epoch 3/10 | train_loss=0.5399 | test_loss=0.4649 | test_acc=0.7636
Epoch 4/10 | train_loss=0.5288 | test_loss=0.4763 | test_acc=0.7636
Epoch 5/10 | train_loss=0.5197 | test_loss=0.4377 | test_acc=0.7636
Epoch 6/10 | train_loss=0.5000 | test_loss=0.4318 | test_acc=0.7636
Epoch 7/10 | train_loss=0.4905 | test_loss=0.4419 | test_acc=0.7636
Epoch 8/10 | train_loss=0.4806 | test_loss=0.4590 | test_acc=0.7636
Epoch 9/10 | train_loss=0.4800 | test_loss=0.3990 | test_acc=0.7636
Epoch 10/10 | train_loss=0.4738 | test_loss=0.3957 | test_acc=0.7636
Saved model -> models/final_model_centralized.pt


wandb: updating run metadata
wandb: uploading output.log
wandb: uploading output.log; uploading config.yaml
wandb: 
wandb: Run history:
wandb:      epoch ▁▂▃▃▄▅▆▆▇█
wandb:   test_acc ▁▁▁▁▁▁▁▁▁▁
wandb:  test_loss █▅▄▅▃▃▃▄▁▁
wandb: train_loss █▆▄▄▃▂▂▁▁▁
wandb: 
wandb: Run summary:
wandb:            epoch 10
wandb:   final_test_acc 0.76364
wandb:  final_test_loss 0.39568
wandb: final_train_loss 0.47383
wandb:         test_acc 0.76364
wandb:        test_loss 0.39568
wandb:       train_loss 0.47383
wandb: 
wandb: 🚀 View run centralized at: https://wandb.ai/vae-seminar/FedCAD/runs/qieo1jzm
wandb: ⭐️ View project at: https://wandb.ai/vae-seminar/FedCAD
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20260121_150146-qieo1jzm/logs



=== Centralized run, seed=1 ===

>>> /Users/zoe/Desktop/Fed-CAD/.venv/bin/python train_central.py --epochs 10 --lr 0.001 --eval-interval 1 --wandb-mode online
    in: /Users/zoe/Desktop/Fed-CAD
    with env overrides: {'WANDB_PROJECT': 'FedCAD', 'WANDB_MODE': 'online', 'WANDB_RUN_GROUP': 'centralized', 'WANDB_TAGS': 'centralized,notebook', 'SEED': 1}


...

wandb: updating run metadata
wandb: updating run metadata; uploading output.log; uploading wandb-summary.json
wandb: uploading config.yaml
wandb: uploading history steps 0-9, summary, console lines 0-10
wandb: 
wandb: Run history:
wandb:      epoch ▁▂▃▃▄▅▆▆▇█
wandb:   test_acc ▁▁▁▁▃▁▆▅▅█
wandb:  test_loss █▆▇▃▃▄▆▄▃▁
wandb: train_loss █▇▆▄▃▃▂▂▁▁
wandb: 
wandb: Run summary:
wandb:            epoch 10
wandb:   final_test_acc 0.82727
wandb:  final_test_loss 0.3996
wandb: final_train_loss 0.46476
wandb:         test_acc 0.82727
wandb:        test_loss 0.3996
wandb:       train_loss 0.46476
wandb: 
wandb: 🚀 View run centralized at: https://wandb.ai/vae-seminar/FedCAD/runs/nbotorv0
wandb: ⭐️ View project at: https://wandb.ai/vae-seminar/FedCAD
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20260121_150157-nbotorv0/logs



=== Centralized run, seed=2 ===

>>> /Users/zoe/Desktop/Fed-CAD/.venv/bin/python train_central.py --epochs 10 --lr 0.001 --eval-interval 1 --wandb-mode online
    in: /Users/zoe/Desktop/Fed-CAD
    with env overrides: {'WANDB_PROJECT': 'FedCAD', 'WANDB_MODE': 'online', 'WANDB_RUN_GROUP': 'centralized', 'WANDB_TAGS': 'centralized,notebook', 'SEED': 2}


wandb: Currently logged in as: zoematr (vae-seminar) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.23.0
wandb: Run data is saved locally in /Users/zoe/Desktop/Fed-CAD/wandb/run-20260121_150209-p35d83mv
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run centralized
wandb: ⭐️ View project at https://wandb.ai/vae-seminar/FedCAD
wandb: 🚀 View run at https://wandb.ai/vae-seminar/FedCAD/runs/p35d83mv


Epoch 1/10 | train_loss=0.6392 | test_loss=0.5580 | test_acc=0.7636
Epoch 2/10 | train_loss=0.6029 | test_loss=0.4999 | test_acc=0.7636
Epoch 3/10 | train_loss=0.5778 | test_loss=0.5063 | test_acc=0.7636
Epoch 4/10 | train_loss=0.5566 | test_loss=0.4806 | test_acc=0.7636
Epoch 5/10 | train_loss=0.5338 | test_loss=0.4339 | test_acc=0.7636
Epoch 6/10 | train_loss=0.5098 | test_loss=0.4260 | test_acc=0.8091
Epoch 7/10 | train_loss=0.5022 | test_loss=0.4384 | test_acc=0.7455
Epoch 8/10 | train_loss=0.4944 | test_loss=0.4167 | test_acc=0.8000
Epoch 9/10 | train_loss=0.4771 | test_loss=0.3976 | test_acc=0.8000
Epoch 10/10 | train_loss=0.4703 | test_loss=0.3957 | test_acc=0.7909
Saved model -> models/final_model_centralized.pt


wandb: updating run metadata
wandb: uploading wandb-summary.json; uploading config.yaml
wandb: uploading history steps 0-9, summary, console lines 0-10
wandb: 
wandb: Run history:
wandb:      epoch ▁▂▃▃▄▅▆▆▇█
wandb:   test_acc ▃▃▃▃▃█▁▇▇▆
wandb:  test_loss █▅▆▅▃▂▃▂▁▁
wandb: train_loss █▆▅▅▄▃▂▂▁▁
wandb: 
wandb: Run summary:
wandb:            epoch 10
wandb:   final_test_acc 0.79091
wandb:  final_test_loss 0.3957
wandb: final_train_loss 0.4703
wandb:         test_acc 0.79091
wandb:        test_loss 0.3957
wandb:       train_loss 0.4703
wandb: 
wandb: 🚀 View run centralized at: https://wandb.ai/vae-seminar/FedCAD/runs/p35d83mv
wandb: ⭐️ View project at: https://wandb.ai/vae-seminar/FedCAD
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20260121_150209-p35d83mv/logs


## Federated experiments (Flower Simulation Engine)

Runs `flwr run .` once per seed. Ensure your Flower app logs to W&B (same project) with metrics comparable to centralized (e.g., `global_train_loss`, `global_test_loss`, `global_test_acc`).

Tip: If your Flower app reads `WANDB_*` env vars and `SEED`, the grouping and seeds below will apply.

In [ ]:
def run_flower(extra_env=None):
    # Prefer CLI if available; fallback to `python -m flwr`
    if shutil.which("flwr"):
        run_cmd(["flwr", "run", "."], extra_env=extra_env, cwd=os.getcwd())
    else:
        run_cmd([sys.executable, "-m", "flwr", "run", "."], extra_env=extra_env, cwd=os.getcwd())

for seed in SEEDS:
    env_overrides = {
        "WANDB_PROJECT": WANDB_PROJECT,
        "WANDB_MODE": WANDB_MODE,
        "WANDB_RUN_GROUP": "federated",
        "WANDB_TAGS": "federated,notebook",
        # Many FL apps use this to control randomness; ensure your app reads it
        "SEED": seed,
    }
    print(f"\n=== Federated run (Flower), seed={seed} ===")
    run_flower(extra_env=env_overrides)
    time.sleep(1)